# Minesweeper: Knowledge Bases & Logical Inference 

By **Rohan Rele** (rsr132), **Aakash Raman** (abr103), **Alex Eng** (ame136), and **Adarsh Patel** (aap237)

This project was completed for Professor Wes Cowan's Fall 2019 offering of the CS 520: Intro to Artificial Intelligence course, taught at Rutgers University, New Brunswick.

## Baseline Algorithm 

We began the assignment by implementing the basic solver agent algorithm that relies solely on local inference and iterative deduction. 

### Implementation 

##### Class Specification

Throughout this assignment we maintained a very object-oriented approach. The baseline case includes two main components. The Agent class and the MineSweeper class. The Agent represents the "player" of the game while the MineSweeper class represents the game as it transitions through its different states. Below are the class details for the two. 

In [1]:
class agent:
    def __init__(self, game, order):
        # copy game into agent object
        self.game = deepcopy(game)
        # track player knowledge per cell: initially all hidden
        self.playerKnowledge = np.array([[HIDDEN] * self.game.dim for _ in range(self.game.dim)])

        # keep track of these throughout game to yield final mineFlagRate
        self.numFlaggedMines = 0
        self.numDetonatedMines = 0

        # these are the two metrics we can judge performance with
        self.mineFlagRate = 0
        self.totalSolveTime = 0
        self.logging = False

        self.order = order
        self.current_in_order = 0

        self.uncertaintyType = 'none'


As we can see, the agent's members include:

1. (Type: Minesweeper) A MineSweeper object, which specifies the current instance of the game that agent is playing on
2. (Type: Numpy Array 2d) An integer matrix that keeps track of what cells have been queried, their *value* (vicinity to a mine) and/or their "mine status"

And the rest are simple metrics used to track the agent's progress: 

3. (Type: Integer) Number of flagged mines 
4. (Type: Integer) Number of detonated mines
5. (Type: Integer) Ratio between mines and flags
6. (Type: Integer) Time taken 
7. (Type: Boolean) Toggle the logger 
8. (Type: List) A fixed (but random) order of cell coordinates that we use to minimize variance of repeated trials 
9. (Type: Integer) Current order ? 
10. (Type: String) Uncertainty toggle for the bonus: when the clue is randomly revealed, optimistic, or cautious. Defaults to 'none' i.e. a revealed clue is accurate.

See <font color=blue>Agent.py</font> for more implementation details.

In [2]:
dirs = [(0, 1), (0, -1), (1, 0), (-1, 0), (1, 1), (-1, -1), (-1, 1), (1, -1)]

MINE = -6
DETONATED = -8
HIDDEN = -1
SAFE = 0

class MineSweeper:
    dim = 0
    num_mines = 0
    board = [[]]
    success = False

    def __init__(self, dim, num_mines):
        self.dim = dim

        if num_mines < 0 or num_mines > dim**2:
            print("Invalid number of mines specified --> set to 0")
            num_mines = 0
        self.num_mines = num_mines

        board = [[0 for _ in range(dim)] for _ in range(dim)]

        for n in range(num_mines):
            x = random.randint(0, dim-1)
            y = random.randint(0, dim-1)
            while board[x][y] == MINE:
                x = random.randint(0, dim-1)
                y = random.randint(0, dim-1)
            board[x][y] = MINE

        temp = np.array(board)

        coords = zip(*np.where(temp == MINE))
        for x, y in coords:
            for i, j in dirs:
                if 0 <= x + i < dim and 0 <= y + j < dim and temp[x+i][y+j] != MINE:
                    temp[x + i][y + j] += 1
                else:
                    continue

        self.board = temp.tolist()


The minesweeper class is even more straightforward and enables the following: 

(I)  . Specifies coordinates to easily iterate through local cells

(II) . Specifies a key for the board (mine, hidden, safe...etc.) 

(III). Places random mines throughout our data structure based on some params (dim = dimension, n = # of mines) 

(IV) . Updates data structure with clues (+= 1 to every surrounding cell of a mine) 

Its members are: 

1. (Type: Integer) Dimension 
2. (Type: Integer) Number of mines 
3. (Type: 2d list) Board 
4. (Type: Boolean) Whether or not the board was solved

See <font color=blue>Minesweeper.py</font> for more implementation details.

##### Solver 

As is the baseline or "naive" approach to minesweeper, our solver algorithm begins with a random first move (i.e. revealing a cell). It then tries to deduce, based solely on the 8 cells adjacent to it, whether or not any of the 9 cells in question are definitely safe or definitely a mine. Clearly, the chosen cell does not need to be inferred, but the 8 cells adjacent can generally not be inferred on the first move *unless* the clue is 0. Mathematically, we can denote this as: $P(X_{i,j} = 0 | C) = (1-d)^9$ where $C$ is the board configuration and $d$ is the mine density (Note: we are assuming the first cell queried is *not* a corner or edge cell, but our implementation handles all cases).  

See <font color=blue>solve(...)</font> method in <font color=blue>Agent.py</font> for more implementation details.

See *attached* <font color=blue>/data/log.txt</font> for full breakdown of moves made by baseline agent. The following snippet illustrates the above methodology. 

BASELINE SOLVER LOG (initial moves) 

Cell (8, 2) safely revealed with clue: 3.
	# safe neighbors: 0
	# mine neighbors: 0
	# hidden neighbors: 8
	# total neighbors: 8


Revealing cell (8, 2) led to no conclusive next move (either DETONATED or all neighbors MINES).

Will attempt to re-deduce & enqueue new safe cell(s) from all of current knowledge,

or add random if none available.

	Re-processing did not find new safe cells; proceeding to randomly select hidden cell.

----------------------------------------

Cell (3, 5) safely revealed with clue: 1.
	# safe neighbors: 0
	# mine neighbors: 0
	# hidden neighbors: 8
	# total neighbors: 8


Revealing cell (3, 5) led to no conclusive next move (either DETONATED or all neighbors MINES).

Will attempt to re-deduce & enqueue new safe cell(s) from all of current knowledge,

or add random if none available.

	Re-processing did not find new safe cells; proceeding to randomly select hidden cell.

----------------------------------------

BOOM! Mine detonated at (5, 5).


Revealing cell (5, 5) led to no conclusive next move (either DETONATED or all neighbors MINES).

Will attempt to re-deduce & enqueue new safe cell(s) from all of current knowledge,

or add random if none available.

	Re-processing did not find new safe cells; proceeding to randomly select hidden cell.

----------------------------------------

Cell (3, 7) safely revealed with clue: 1.
	# safe neighbors: 0
	# mine neighbors: 0
	# hidden neighbors: 8
	# total neighbors: 8


Revealing cell (3, 7) led to no conclusive next move (either DETONATED or all neighbors MINES).

Will attempt to re-deduce & enqueue new safe cell(s) from all of current knowledge,

or add random if none available.

	Re-processing did not find new safe cells; proceeding to randomly select hidden cell.

----------------------------------------

Cell (11, 7) safely revealed with clue: 0.
	# safe neighbors: 0
	# mine neighbors: 0
	# hidden neighbors: 8
	# total neighbors: 8


All neighbors of (11, 7) must be safe.

	Neighbor (11, 8) flagged as safe and enqueued for next visitation.

	Neighbor (11, 6) flagged as safe and enqueued for next visitation.

	Neighbor (12, 7) flagged as safe and enqueued for next visitation.

	Neighbor (10, 7) flagged as safe and enqueued for next visitation.

	Neighbor (12, 8) flagged as safe and enqueued for next visitation.

	Neighbor (10, 6) flagged as safe and enqueued for next visitation.

	Neighbor (10, 8) flagged as safe and enqueued for next visitation.

	Neighbor (12, 6) flagged as safe and enqueued for next visitation.

----------------------------------------

...


When enough of the board has been filled, the metrics of each cell start to be the main source of knowledge for the agent. At every cell $(x,y)$, the agent keeps track of $(x,y)$'s total neighbors, its safe neighbors, mine neighbors and hidden neighbors, which can be used for marking more cells as safe/unsafe. For example, if a cell has 8 total neighbors, 7 safe neighbors, 1 hidden neighbor and a clue of 1, the agent can determine with 100% certainty that the final neighbor is a mine. Once the agent can no longer make any reliable inferences, it returns to the last safe, visited cell which is stored in a queue. If the queue is empty, the agent returns to random selection

Below is a snippet from <font color=blue>/data/log.txt</font> illustrating this more concretely. 

BASELINE SOLVER LOG (intermediate moves) 

...

----------------------------------------

Cell (6, 5) safely revealed with clue: 4.
	# safe neighbors: 2
	# mine neighbors: 2
	# hidden neighbors: 4
	# total neighbors: 8


Revealing cell (6, 5) led to no conclusive next move (either DETONATED or all neighbors MINES).

Will attempt to re-deduce & enqueue new safe cell(s) from all of current knowledge,

or add random if none available.

	Re-processing KB found that: All neighbors of (2, 5) must be safe.

		Neighbor (1, 6) flagged as safe and enqueued for next visitation.

----------------------------------------

Cell (1, 6) safely revealed with clue: 3.
	# safe neighbors: 4
	# mine neighbors: 1
	# hidden neighbors: 3
	# total neighbors: 8


Revealing cell (1, 6) led to no conclusive next move (either DETONATED or all neighbors MINES).

Will attempt to re-deduce & enqueue new safe cell(s) from all of current knowledge,

or add random if none available.

	Re-processing KB found that: All neighbors of (0, 5) must be mines.

		Neighbor (0, 6) flagged as a mine.

	Re-processing did not find new safe cells; proceeding to randomly select hidden cell.

----------------------------------------

Cell (12, 4) safely revealed with clue: 5.
	# safe neighbors: 0
	# mine neighbors: 0
	# hidden neighbors: 8
	# total neighbors: 8


Revealing cell (12, 4) led to no conclusive next move (either DETONATED or all neighbors MINES).

Will attempt to re-deduce & enqueue new safe cell(s) from all of current knowledge,

or add random if none available.

	Re-processing did not find new safe cells; proceeding to randomly select hidden cell.

----------------------------------------
...

*All iterations of <font color=blue>/data/log.txt</font> were from the following game (refer to key for integer meanings)* 

<table>
    <tr>
        <td>
<figure>
    <img src='./imgs/baselineTrialRun_init_board.png' width="400" height="200" alt='missing' />
    <figcaption>Initial Board (Unhidden)</figcaption>
</figure>
        </td>
        <td>
<figure>
    <img src='./imgs/baselineTrialRun_solved_board.png' width="400" height="200" alt='missing' />
    <figcaption>Solved Board</figcaption>
</figure>
        </td>
    </tr>
</table>

### Performance & Results 

The following tables show how the baseline algorithm performs under different parametrized conditions.

In [2]:
import pandas as pd 

#### Baseline algorithm: Average mine flag rates over various board sizes and mine densities

In [3]:
baselineFlagRates = pd.read_csv("./data/baseline_solo_performance_flagRates.csv", index_col=0)
baselineFlagRates

,d=0.1,d=0.15,d=0.2,d=0.25,d=0.3,d=0.5
dim=10,0.971500,0.939333,0.865000,0.802200,0.729667,0.526600
dim=20,0.995500,0.981833,0.942125,0.864100,0.785250,0.555425
dim=30,0.999722,0.993630,0.962444,0.887156,0.801241,0.562000
dim=40,1.007125,1.002208,0.975156,0.906350,0.815563,0.563713
dim=50,1.008880,1.005120,0.982620,0.910544,0.822400,0.571800



This table depicts the average flag rates (detected mines/total mines) per dim/density and clearly we can see a strictly decreasing trend as the mine density increases. This is because more mines not only forces the baseline agent to make more random choices but also makes it more prone to choosing mines on those random iterations.

*Note:* Some of the flag rates are $>1$ very slightly, this could be because of average rounding errors. Each cell in the table is an average value of 100-200 trials. 

# TODO: possible scatter plot?

#### Baseline algorithm: Average runtime over various board sizes and mine densities

In [4]:
baselineTimes = pd.read_csv("./data/baseline_solo_performance_times.csv",index_col=0)
baselineTimes

,d=0.1,d=0.15,d=0.2,d=0.25,d=0.3,d=0.5
dim=10,0.008125,0.011161,0.018602,0.020182,0.023605,0.027591
dim=20,0.041218,0.064340,0.146992,0.208602,0.288224,0.366105
dim=30,0.095811,0.192290,0.501797,0.964674,1.265326,1.775726
dim=40,0.181584,0.447265,1.297337,2.770875,3.816501,5.791174
dim=50,0.273127,0.701709,2.810102,6.401690,8.937176,13.768566


This table depicts the average performance times (in seconds) per dim/density. Again, we test each pairing 100-200 times and average the runtimes of the trials. Like the previous table, the pattern is evident; higher dims/densities correspond to longer runtimes due to more random choicing and recomputation of the knowledge base. 

# TODO: possible scatter plot?

## Improved Algorithm 

We decided on an approach that relies not solely on local inference but evaluates all the clues given to evaluate the whole board at every iteration. We will refer to this as the **Linear Algebra Approach**. 

### Implementation 

##### Representation 

Our Linear Algebra Agent (*see <font color=blue>LinAlg.py</font>*) inherits all members & methods from our generic Agent class, and thus its object representation is nearly identical (except one extra metric for computational convenience); as is the board structure that it operates on. 

In [5]:
class lin_alg_agent(agent):
    def __init__(self, game, useMineCount, order):
        agent.__init__(self,game, order)
        self.useMineCount = useMineCount

##### Inference 

Rather than performing inference based on a restricted set of clues/information about adjacent data, the Linear Algebra appraoch does inference on the entire board, which effectively expands the knowledge base by a factor of $(dim-3)^2$. It then uses that expanded knowledge base to create a constraint satisfaction problem or, more conceretly, a system of linear equations that analytically describes what cells are *guaranteed* to be mines. 

This is illustrated below:

<table>
    <tr>
        <td>
<figure>
    <img src='./imgs/ss1.png' width="150" height="650" alt='missing' />
    <figcaption>Example block of game board with variable labels</figcaption>
</figure>
        </td>
        <td>
<figure>
    <img src='./imgs/ss2.png' width="300" height="150" alt='missing' />
    <figcaption>Corresponding System of Equations</figcaption>
</figure>
        </td>
  <td>
<figure>
    <img src='./imgs/ss4.png' width="300" height="150" alt='missing' />
    <figcaption>Corresponding Matrix</figcaption>
</figure>
        </td>
    </tr>
</table>

*Images from https://massaioli.wordpress.com/2013/01/12/solving-minesweeper-with-matricies/comment-page-1/*

##### Decisions 

Decision making is fairly straightforward, once the matrix is row reduced, we can mark cells that correspond to reduced rows with a 0 in the final column vector as being safe and similarly reduced rows with value 1 in the final column vector as mine. Unreduced or 0 row vectors are disregarded. 

The Linear Algebra Agent's decision-making process is logged out in a snippet below (see <font color=blue>/data/lin_alg_log.txt</font>):

...

----------------------------------------

Cell (2, 2) safely revealed with clue: 3.
	# safe neighbors: 4
	# mine neighbors: 1
	# hidden neighbors: 3
	# total neighbors: 8


Revealing cell (2, 2) led to no conclusive next move (either DETONATED or all neighbors MINES).

Will attempt to re-deduce & enqueue new safe cell(s) from all of current knowledge,

or add random if none available.

game:

\[0, 0, 0, 0]

[1, 1, 1, 1]

[-6, 2, 3, -6]

[2, -6, 3, -6]


knowledge:

[[ 0  0  0  0]

 [ 0  0  0  0]
 
 [-6  0  0 -6]
 
 [-1 -1 -1 -1]]


generated following row using (2,1): 

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 1.]

generated following row using (2,2): 

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 2.]

generated following row using total mine count: 

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 2.]

information matrix:

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 1.]

 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 2.]
 
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 2.]]

rref'd matrix:

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]

 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1.]
 
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]]
 

using row: 

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]

deduced (3,0) to be safe via lin alg

using row: 

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1.]

using row: 

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]

deduced (3,3) to be a mine via lin alg

----------------------------------------

Cell (3, 0) safely revealed with clue: 2.
	# safe neighbors: 1
	# mine neighbors: 1
	# hidden neighbors: 1
	# total neighbors: 3


All neighbors of (3, 0) must be mines.

	Neighbor (3, 1) flagged as a mine.

Revealing cell (3, 0) led to no conclusive next move (either DETONATED or all neighbors MINES).

Will attempt to re-deduce & enqueue new safe cell(s) from all of current knowledge,

or add random if none available.

game:

[0, 0, 0, 0]

[1, 1, 1, 1]

[-6, 2, 3, -6]

[2, -6, 3, -6]


knowledge:

[[ 0  0  0  0]

 [ 0  0  0  0]
 
 [-6  0  0 -6]
 
 [ 0 -6 -1 -6]]
 

generated following row using (2,1): 

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]

generated following row using (2,2): 

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]

generated following row using total mine count: 

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]

information matrix:

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]

 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]


rref'd matrix:

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]

 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
 

using row: 

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]

deduced (3,2) to be safe via lin alg

using row: 

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

using row: 

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

----------------------------------------

Cell (3, 2) safely revealed with clue: 3.
	# safe neighbors: 2
	# mine neighbors: 3
	# hidden neighbors: 0
	# total neighbors: 5


All neighbors of (3, 2) are already revealed; nothing to infer.

----------------------------------------
...

*The game board that yielded the above log is displayed here:*

<table>
    <tr>
        <td>
<figure>
    <img src='./imgs/linalgboard.png' width="300" height="150" alt='missing' />
    <figcaption>Game board (unhidden)</figcaption>
</figure>
        </td>
        <td>
<figure>
    <img src='./imgs/linalgknowledge.png' width="300" height="150" alt='missing' />
    <figcaption>Agent Solution at final state</figcaption>
</figure>
        </td>
        
   </tr>
</table>

Notice that in the final state, there is one block still HIDDEN, however the agent terminated its gameplay because the number of mines is in its knowledge base. Had the number of mines not been specified, the agent would have continued until all the remaining hidden cells were revealed, adding slightly computation (in this low-dimensional case). In higher dimensions, these extra computation could be sizeable though. 

### Improvements, Performance & Results 

Below is a comparison table of the Linear Algebra approach with the baseline approach in terms of mine detection rate (once again, DETECTED/total) as a percentage and performance time in seconds.


In [6]:
df = pd.read_excel("./data/comparisondata.xlsx",sheet_name=0,header=0).dropna()
df.style.hide_index()

Dim,Density,Baseline Time,Lin alg time,Baseline %,Lin Alg %
10,0.1,0.01,0.01,96.3,97.3
10,0.2,0.02,0.03,85.9,91.5
10,0.3,0.02,0.07,72,79.8
10,0.5,0.03,0.1,53.2,57.6
20,0.1,0.04,0.07,99.2,99.4
20,0.2,0.15,0.31,93.8,97.2
20,0.3,0.34,1.88,78.2,85.3
20,0.5,0.45,4.81,55.3,59.4
30,0.1,0.12,0.33,99.6,99.7
30,0.2,0.67,1.64,95.5,98.3


# TODO: need to use updated data (./data/comparison data updated xlsx) & graph this: he wants mine density versus flag rates (baseline vs. linalg) for various trials of fixed dim


The above data is representative of 50-100 trials (depending on dim/density) which were averaged together. We can see that using the linear algebra approach consistently yields an improvement from the baseline case. Performance time, of course, is also strictly increasing; dramatically so when the mine density is high, since the system of linear equations becomes much larger exponentially and thus forces the agent to perform more computations. 


## Improved Algorithm v 1.5

This is a modified version of the **Linear Algebra approach** that makes improvements to guessing. Any algorithm to solve Minesweeper will have to eventually make a random guess. The improvements outlined in this version of our approach attempts to address this problem and instead make an *educated guess* using a combination of brute force and probabilistic inference 

### Implementation 

The Brute Force Agent inherits all of the Linear Algebra Agent's (and thus, the Generic Agent's) members/functions. The object-oriented schema remains unchanged throughout this assignment. 

In [7]:
class brute_force_agent(lin_alg_agent):
    def __init__(self, game, useMineCount, order):
        lin_alg_agent.__init__(self,game,useMineCount,order)

The main advantage of adding brute force functionality is that upon failure of the Linear Algebra Agent's main method (i.e. solving the system of equations via matrix algebra), it no longer has to randomly guess what the next move will be. Instead, we iterate through all possible configurations (of relevant cells) and keep track of how many times a mine appeared in each cell. We then simply choose the cell with the least probability of having a mine. 

The methods that perform these computations are the configuration generation function : 

In [8]:
def get_configs(self, configCells, consistency_cells):
        configs = [(deepcopy(self.playerKnowledge), i, [])  for i in range(len(configCells))]
        out = []
        while configs:
            next_set = []
            for board, index, currentConfig in configs:
                current_board = deepcopy(board)
                current_config = deepcopy(currentConfig)
                dim = self.game.dim

                if index >= len(configCells) or (self.useMineCount and len(current_config) == self.game.num_mines-self.numFlaggedMines-self.numDetonatedMines):
                    if self.confirm_full_consistency(current_board, consistency_cells):
                        out.append(current_config)
                    continue
                cell = configCells[index]
                current_config.append(cell)
                x = cell[0]
                y = cell[1]
                current_board[x,y] = MINE
                valid = True
                for dx, dy in dirs:
                    nx = x + dx
                    ny = y + dy
                    if 0 <= nx < dim and 0 <= ny < dim and self.playerKnowledge[nx][ny] == SAFE and not self.confirm_consistency(current_board, nx, ny):
                        valid = False
                if valid:
                    for i in range(index + 1, len(configCells) + 1):
                        next_set.append((current_board,i,current_config))

            configs = next_set
        return out

and the probability calculating function: 

In [9]:
def probability_method(self):
        # print(self.playerKnowledge)
        dim = self.game.dim
        consistency_cells = []
        config_cells = list()
        for x in range(dim):
            for y in range(dim):
                if self.playerKnowledge[x][y] == SAFE:
                    numSafeNbrs, numMineNbrs, numHiddenNbrs, numTotalNbrs = self.getCellNeighborData(x, y)
                    if numHiddenNbrs > 0:
                        consistency_cells.append((x,y))
                        children = set(self.get_hidden_neighbors(x,y))
                        # print(children)
                        intersects = []
                        for i,s in enumerate(config_cells):
                            if len(children.intersection(s)) > 0:
                                intersects.append(i)
                        if len(intersects) > 0:
                            for i in intersects:
                                children.update(config_cells[i])
                            for i in intersects[::-1]:
                                del config_cells[i]
                        config_cells.append(children)
        #                 print(config_cells)
        # print(config_cells)

        if len(consistency_cells) == 0:
            return self.get_next_random(set())

        config_cells = [sorted(list(y), key = lambda x: x[0] * dim + x[1]) for y in config_cells]

        configs = []
        to_remove = []
        for i,s in enumerate(config_cells):
            if len(s) > 20:
                to_remove.append(i)
                # print("len(s)={}, ignoring".format(len(s)))
                continue
            start_search_time = time.time()
            configs.append(self.get_configs(s, consistency_cells))
            if time.time() - start_search_time > 10:
                print("len(s)={}, took {} seconds to compute".format(len(s), round(time.time() - start_search_time,2)))
        # print([len(s) for s in config_cells])
        for i in to_remove[::-1]:
            del config_cells[i]
        # print([len(s) for s in config_cells])
        # print(len(config_cells))
        # print(len(configs))
        if len(configs) == 0:
            return self.get_next_random(set())


        min_mine_count = sum([ 0 if len(s) == 0 else min([len(config) for config in s]) for s in configs])
        for s in configs:
            min_for_set =  0 if len(s) == 0 else min([len(config) for config in s])
            to_remove = []
            for i,config in enumerate(s):
                if len(config) + min_mine_count - min_for_set > self.game.num_mines-self.numFlaggedMines-self.numDetonatedMines:
                    to_remove.append(i)
            for i in to_remove[::-1]:
                del s[i]

        # print("found configs:")
        probabilities = dict()
        for i,s in enumerate(configs):
            if len(s) > 0:
                counts = {x:0 for x in config_cells[i]}
                for config in s:
                    for coordinates in config:
                        counts[coordinates] += 1
                for k , v in counts.items():
                    probabilities[k] = v / len(s)
            else:
                for cell in config_cells[i]:
                    probabilities[cell] = 0

        best_cell = None
        best_probability = len(configs)

        for cell, probability in probabilities.items():
            if probability <= best_probability:
                best_cell = cell
                best_probability = probability

        other_cells = self.numHiddenCells() - len(config_cells)
        mines_left = self.game.num_mines-self.numFlaggedMines-self.numDetonatedMines
        max_mines_in_hidden_cells_not_in_config_Cells = mines_left - min_mine_count
        other_cell_max_probability = 1 if other_cells == 0 else max_mines_in_hidden_cells_not_in_config_Cells / other_cells

        if best_probability < other_cell_max_probability:
            assert best_cell is not None
            return best_cell
        else:
            to_exclude = []
            for l in config_cells:
                to_exclude.extend(l)
            return self.get_next_random(set(to_exclude))

See <font color=blue>BruteForceAgent.py</font> for more implementation details.

### Improvements, Performance & Results 

# TODO: add in updated data (like in above section). need to graph this: he wants mine density versus flag rates (baseline vs. linalg + brute) for various trials of fixed dim

## Bonus - Dealing with Uncertainty 

This section dealt with a variation of the MineSweeper game in which the clue itself is not assumed to be correct. That is, we deal with uncertainty on the accuracy of the clue itself. Accordingly, instead of merely revealing the actual clue via `clue = self.game.board[x][y]`, we now use the `getClue(...)` function found in `Agent.py` which returns an uncertain clue. Note that toggling between these types of uncertain clues, as well as having no uncertainty (as before), is done via the `self.uncertaintyType` attribute in the `Agent` class.

### Accurate information, random reveal

In this case, the revealed clue is accurate, but it is only revealed with some random probability $p$. This complicates the game because the knowledge base is not updated as frequently as before, leading to less opportunities for inference to deterministically detect mines or safe cells.

#### Uncertainty implementation

For the random reveal type of uncertainty, the `getClue` function needs to be passed $p$, the probability that any given clue will be revealed. Then, the code to reveal the clue randomly is simple:

In [ ]:
if random.random () < p:
    return self.game.board[x][y]
else:
    return -1

where -1 is returned in place of an actual clue. The driver is coded such that if it receives a -1 clue, it will simply continue.

#### Performance hit

We compare Random Reveal against the baseline strategy to get a sense of how the mine flag rate reacts to this uncertainty. For the following 200 trials, we use Random Reveal with $p = 0.02$; that is, a significantly small probability of revealing the clue.

<figure>
    <img src='./imgs/baseline_randomReveal.png' alt='missing' />
</figure>

We see that, expectedly, the agent is uniformly worse with the random reveal level of uncertainty, with flag rates approximately 40% lower than baseline, on average. As mentioned earlier, the agent has strictly less information in its knowledge base at any point in time than it would have at that step without any uncertainty measures.

Note also that we used $p=0.02$, a very small probability, because while higher probabilities tested in the range $[0.3,0.7]$ also resulted in strictly worse flag rates, they did not produce dramatic disparities as seen here. We found that it takes a relatively small $p$ value in order to significantly affected the flag rates.

#### Strategy adaptation

# TODO: needs work

We first need to assume that we are provided with the information of the number of mines given in the grid. Without this information, it becomes incredibly difficult to estimate the probability. The problem with this situation is that we cannot trust any of our knowledge base, since our clues are all based on probabilities. 

What we could do is that we can using probabilistic inference here. But the problem remains since there is still no way of knowing if our knowledge base will consistency be correct. And if it is not correct, then the game will typically not last too long since mine detection will be quite off from the baseline approach where the clues are given to you with full certainty. 

### Flawed Knowledge-Base: Optimistic clue

#### Uncertainty implementation

The `getClue` function will return an integer uniformly at random from the range $[m, C]$, where $m$ is the number of mine neighbors already revealed, i.e. the minimum number of possible total mine neighbors, and $C$ is the true clue value. That way, the returned value will be somewhere in the range of possible values but always underestimating the true clue.

In [ ]:
if numMineNbrs < self.game.board[x][y]:
    return random.randint(numMineNbrs, self.game.board[x][y])
elif numMineNbrs == self.game.board[x][y]:
    return self.game.board[x][y]

#### Performance hit

<figure>
    <img src='./imgs/baseline_optimistic.png' alt='missing' />
</figure>

We see that the agent acting on optimistic clue knowledge is uniformly worse than the baseline agent, with flag rates approximately 50% lower. This is because with clues which possibly reveal fewer mine neighbors, the agent is unable to deduce as many deterministically mine neighbors. It also may incorrectly determine some cells are safe, because the clue is an underestimate. This fact holds true using the linear algebra + brute force approach as well.

#### Strategy adaptation

Similarly to our linear algebra discussion earlier, consider the equation $x_1 + x_2 + \dots + x_n \geq C$, where $x_i$ is a cell which is 1 if mine, 0 if safe, and $C$ is the given clue. 

Then we naturally see that $\sum_{i}x_i$ gives the true number of mines, and $C$ underestimates this: representing exactly this case of **optimistic clues.**

Leveraging this, we turn our discussion of linear algebra to the discussion of [linear optimization,](https://en.wikipedia.org/wiki/Linear_programming) which is the related field considering systems of linear equations where each equation represents not an equality but a constraint ($\geq$ or $\leq$), and a solution is optimal if it maximizes or minimizes a certain objective function of the variables $z = f(x_i)$.

Because we utilize systems of linear equations in the linear algebra method above, we see it convenient to adapt this portion to a linear optimization model. 

##### Linear optimization theory & implementation

We write another linear optimization agent which inherits from the linear algebra + brute force agent above, with the following simple change: **Instead of finding a solution to the system, we are trying to find the optimal one.** If none can be found, then simply revert to solving the system as before, regardless of the clues' accuracy.

See `LinOpt.py` for more implementation details.

For our optimal clue case, we are considering systems where each equation is of the form: $x_1 + x_2 + \dots + x_n \geq C$. The linear programming problem we use is:

Max $z = \sum_{i}x_i$ subject to constraints $Ax \geq b, x_i \geq 0$.

Here, the expression $Ax \geq b$ is nothing more than the system of equations laid out in a matrix with $\geq$ symbols instead of $=$ symbols.

The primary method of solving linear programming problems is the [simplex method,](https://personal.utdallas.edu/~scniu/OPRE-6201/documents/LP4-Simplex.html) which takes in the augmented matrix $[A | b]$ with a row added to represent the objective function, called the **initial tableau.** Then, it iteratively uses row operations to return a matrix with the solution to the problem. 

However, the simplex method requires initial tableaus be converted to a specific [canonical form:](https://www.usna.edu/Users/math/wakefiel/_files/documents/sa305/notes/note3-25.pdf)

Max $z$ s.t. $Ax = b, x_i \geq 0$.

We use the [Two Phase Simplex Method](http://www.maths.qmul.ac.uk/~ffischer/teaching/opt/notes/notes8.pdf) here to solve this system. Without diving too much into linear optimization theory, this requires us to pass in a different augmented matrix to mediate the problem of having $\geq$ instead of $=$ symbols. Then, we solve the optimization problem and return the appropriate solution matrix. The below code walks through this process of transforming and solving, but see the `simplexOptimistic(...)` method in `LinOpt.py` for more implementation details.

In [ ]:
# optimistic matrix: has inequalities [row] >= clue-numMineNbrs
def simplexOptimistic(matrix):
    # Phase I: solve for artificial variables
    # concatenate -I_n to right (without last col) for slacking vars to resolve >= into =
    tableau = np.column_stack((matrix[:, :-1], -1*np.identity(matrix.shape[0], dtype=int)))
    # concatenate I_n to right (without last col) for artificial vars (to get init basic fsbl soln)
    tableau = np.column_stack((tableau, np.identity(matrix.shape[0])))
    # concatenate back the last col
    tableau = np.column_stack((tableau, matrix[:, -1]))
    # add obj row: max -(all artificial vars)
    obj_row = np.concatenate(([0]*(matrix.shape[1]-1+matrix.shape[0]), [1]*matrix.shape[0], [0]))
    tableau = np.row_stack((tableau, obj_row))
    # row reduce obj row (art vars basic)
    tableau = makeBasic(tableau)
    # solve Phase I with simplex
    tableau = simplex(tableau)
    # Phase I fail: if z != 0, then no optimal solution exists
    if tableau is None or tableau[-1, -1] != 0:
        return None

    # Phase II: update and simplex
    # delete artificial vars' columns
    tableau = np.column_stack((tableau[:, :(matrix.shape[1]-1 + matrix.shape[0])], tableau[:, -1]))
    # update objective function: all -1s except for slacking vars
    obj_row = np.concatenate(([-1]*(matrix.shape[1] - 1), [0]*(matrix.shape[0] + 1)))
    tableau[-1] = obj_row
    # row reduce to get basic vars std col vectors
    tableau = makeBasic(tableau)
    # simplex as usual
    tableau = simplex(tableau)

    # if sol'n, cut out cols for slackvars and obj row (only returning row vals as in rref)
    if tableau is not None:
        tableau = np.column_stack((tableau[:, :matrix.shape[0]], tableau[:, -1]))[:-1, :]
    return tableau

This of course relies on the `simplex(...)` method to carry out the necessary iterative algorithmic steps:

In [ ]:
def simplex(tableau):
    i = 0
    while (not checkOptimal(tableau)):
        e = enteringVar(tableau)
        d = departingVar(tableau,e)
        tableau = rowReduce(tableau,d,e)
        if tableau is None or hasNoSolution(tableau):
            return None
        i+=1
        # hardcode short-circuit if we get stuck in degeneracy / bland's rule cycling issues
        if i > 20000:
            return None
    return tableau

To see more of how we implemented the linear optimization operations (e.g. row operations, selecting departing/entering variables, handling termination conditions and cycling) behind this, see the `simplex(...)` and related methods in `LinOpt.py`.

##### Trial results

Here is a test run of the linear optimization agent (on a 10 by 10 board with 20 mines) attempting to mediate optimistic clues with the Two Phase Simplex Method:

--------------

Solving with LINEAR OPTIMIZATION strategy

Uncertainty: optimistic

Cell (1, 5) safely revealed with clue: 1.
	# safe neighbors: 0
	# mine neighbors: 0
	# hidden neighbors: 8
	# total neighbors: 8


Revealing cell (1, 5) led to no conclusive next move (either DETONATED or all neighbors MINES).

Will attempt to re-deduce & enqueue new safe cell(s) from all of current knowledge,

or add random if none available.

...

generated following row using total mine count: 
\[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.  1.  1. 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. 20.]

Linear optimization for optimistic uncertainty FAILED. Proceeding with regular Lin Alg.

solved matrix:
[[ 1.  1.  1.  1.  0.  0.  0.  1.  1.  1.  1.  1.  1.  1.  0.  0.  0.  1. 1.  1.  1.  1.  1.  1.  0.  0.  0.  1.  1.  1.  1.  1.  1.  1.  1.  1.   1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.   1.  1.  1.  1. 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.   1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.   1.  1.  1.  1.  1.  1.  1.  1.  1.  1. 17.]

[ 0.  0.  0.  0.  1.  1.  1.  0.  0.  0.  0.  0.  0.  0.  1.  0.  1.  0. 0.  0.  0.  0.  0.  0.  1.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  3.]]
   
...


***** GAME OVER *****

Game ended in 2.826066255569458 seconds

Safely detected (without detonating) 35.0% of mines

-----------------------------------------

Here and throughout the log (which can be read in full at `/data/optimisticLog.txt`), we see that the Two Phase Simplex Method failed to find an optimal solution of equations, and reverted to Lin Alg approach. In this case, our strategy is unable to overcome the significant performance hit of the inaccurate (optimistic) clue, with a low flag rate of 35%.

Before analyzing why this failure occurs, we proceed to also implement a similar solution for cautious clues.

### Flawed Knowledge-Base: Cautious clue

#### Uncertainty implementation

The `getClue` function will return an integer uniformly at random from the range $[C, M]$, where $C$ is the true clue as before, and $M$ is the number of mine neighbors already revealed plus the number of hidden neighbors, i.e. the maximum number of possible total mine neighbors. That way, the returned value will be somewhere in the range of possible values but always overestimating the true clue.

In [ ]:
if self.game.board[x][y] < numMineNbrs + numHiddenNbrs:
    return random.randint(self.game.board[x][y], numMineNbrs + numHiddenNbrs)
elif self.game.board[x][y] == numMineNbrs + numHiddenNbrs:
    return self.game.board[x][y]

#### Performance hit

<figure>
    <img src='./imgs/baseline_cautious.png' alt='missing' />
</figure>

We see that the agent acting on optimistic clue knowledge is uniformly worse than the baseline agent, but in a different way: one should immediately see that the mine flag rates are absurdly above 1.0. This is because the given clues possibly reveal more mine neighbors than there really are, and so the agent is trigger-happy in terms of deducing that certain cells are deterministically mines when in fact they are not. Then, the agent doesn't uncover those cells, even if they are safe, jacking the mine flag rate above 100%.

Now that all three complications have been implemented, we can compare our baseline algorithm's performance hit in each case:

<figure>
    <img src='./imgs/baseline_all.png' alt='missing' />
</figure>

One can see that each successive complication is successively and strictly worse, if we assume that overestimating mines is the worst case scenario of them all. That is, cautious clues are worse than optimistic clues, which are in turn worse than random reveals and no uncertainty at all.

#### Strategy adaptation

With cautious clues, we are concerned of equations of the form: $x_1 + x_2 + \dots + x_n \leq C$. Similarly to the above case with optimistic clues, we use the simplex method, although it is simpler in the sense that constraints having $\leq$ signs is easier to remediate than if they have $\geq$ signs.

See the `simplexCautious(...)` and related methods in `LinOpt.py` for more implementation details.

In [ ]:
# cautious matrix: has inequalities [row] <= clue-numMineNbrs
def simplexCautious(matrix):
    # concatenate I_n to right (without last col) for slacking vars to resolve <= into =
    tableau = np.column_stack((matrix[:, :-1], np.identity(matrix.shape[0])))
    # concatenate back the last col
    tableau = np.column_stack((tableau, matrix[:, -1]))
    # add row for objective function: all -1s except for slacking vars
    obj_row = np.concatenate(([-1]*(matrix.shape[1] - 1), [0]*(matrix.shape[0]+1)))
    tableau = np.row_stack((tableau, obj_row))

    # simplex as usual
    tableau = simplex(tableau)

    # if sol'n, cut out cols for slackvars and obj row (only returning row vals as in rref)
    if tableau is not None:
        tableau = np.column_stack((tableau[:, :matrix.shape[0]], tableau[:, -1]))[:-1, :]
    return tableau

##### Trial results

Just as with the optimistic clue, we run into the same issue where the simplex method fails. Below is a sample from a trial run (similar to before), and the full log can be read in `/data/cautiousLog.txt`.

--------------

Solving with LINEAR OPTIMIZATION strategy

Uncertainty: cautious

BOOM! Mine detonated at (9, 2).


Revealing cell (9, 2) led to no conclusive next move (either DETONATED or all neighbors MINES).

Will attempt to re-deduce & enqueue new safe cell(s) from all of current knowledge,

or add random if none available.

...

generated following row using total mine count: 

\[ 1.  1.  1.  1.  0.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. 1.  1.  1.  1.  1.  1.  1.  1.  1.  0.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.  1.  1.  1.  1.  1.  1.  1. 17.]


Linear optimization for cautious uncertainty FAILED. Proceeding with regular Lin Alg.
solved matrix:

\[[ 1.  1.  1.  1.  0.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. 1.  1.  1.  1.  1.  1.  1.  1.  1.  0.  1.  1.  0.  0.  1.  1.  1.  1.   1.  1.  1.  1.  0.  0.  1.  1.  1.  1.  1.  1.  1.  1.  0.  0.  1.  1.   1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.   1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.   1.  1.  0.  1.  1.  1.  1.  1.  1.  1. 16.]

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]]

...


***** GAME OVER *****

Game ended in 0.46354103088378906 seconds

Safely detected (without detonating) 170.0% of mines

--------------------------------

As discussed earlier, note that the simplex method procedure fails, and the cautious clue results in a hyperactive mine flag rate of 170% due to false positives.

### Implementation challenges for linear optimization

This method (for both optimistic and cautious clues) fails precisely because our problem is not well-suited to the mathematical reality of linear optimization.

Linear optimization theory rests upon a very precise set of geometric conditions. Imagine plotting each constraint inequality, and the region bounded by all these lines. For our system of linear Minesweeper equations, would this region ever exist? What would it look like? In fact, every linear programming problem must have a feasible and convex region of points, i.e. the [feasible region](https://en.wikipedia.org/wiki/Feasible_region), which satisfies the conditions presented by the constraints; without this, the problem will not return a solution. It does not appear that the systems of linear equations we are solving yield such a feasible region, given the failures of our trials.

Moreover, another issue may lie in our objective function: to maximize the total number of mines detected at each pass. This seems right in the sense of intelligent decision-making, but does not correspond to mathematical reality.
Consider the case of optimistic clues. Intuitively, if each constraint is uses a $\geq$ symbol and the objective function is to maximize a certain value, then it would appear that even if there was a feasible region, then there wouldn't be an optimal solution; one might be able to increase the objective function infinitely. 

To fix this, we would have to re-think both of these points. We might end up needing to implement a different method of getting a system of linear equations, e.g. one which solves for safe cells, and not mine cells. Then the system would want to maximize the number of safe cells detected at each pass, because those cells give us new actionable information for inference.

We might also change our objective function to minimize the number of detected mines, to be as conservative as possible. That would mean trying to define the number of detected mines as a function of the mine cells in the system of linear equations used here, or perhaps a modified system.

We might even want to explore looking at the implications of the theory of [duality,](http://web.mit.edu/15.053/www/AMP-Chapter-04.pdf) finding dual problems to the ones at hand, and attempting to solve those.

All in all, linear optimization seems to head in the right intuitive direction, but either is not well-suited to solving Minesweeper with uncertain clues, or needs significant massaging which would depart from the core of the linear algebra systems relayed here.